In [ ]:
API_KEY = '67c1b98b5edbb9d798ac60a8d13f271e'
USER_AGENT = 'Project'

In [ ]:
import requests

headers = {
    'user-agent': USER_AGENT
}

payload = {
    'api_key': API_KEY,
    'method': 'track.search',
    'format': 'json'
}

r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
r.status_code

200

In [ ]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [ ]:
import pandas as pd
df = pd.read_csv('/content/toptags_tracks_file.csv')


In [ ]:
df = df.drop_duplicates(subset=['artist'], keep = 'first')

In [ ]:
df

,tag,track_name,artist
0,rock,Smells Like Teen Spirit,Nirvana
1,rock,Mr. Brightside,The Killers
2,rock,Creep,Radiohead
4,rock,Feel Good Inc.,Gorillaz
5,rock,Yellow,Coldplay
...,...,...,...
2475,new wave,Blue Monday,New Order
2477,new wave,Down Under,Men at Work
2478,new wave,Come on Eileen,Dexys Midnight Runners
2484,new wave,Shut Up and Dance,Walk the Moon


In [ ]:
album_df = pd.DataFrame()

for x in df['artist']:
  result = {}
  r = lastfm_get({
      'method': 'artist.getTopAlbums',
      'artist': x,
      # 'limit':  10
      # 'track': 'Blank Space'
  })

  def jprint(obj):
    global result
    result = obj
    result = r.json()

  jprint(r.json())

  def most_played_album(result):
    global top_tracks_df
    playcount = [i['playcount'] for i in result['topalbums']['album']]
    try:
      mbid = [i['artist']['mbid'] for i in result['topalbums']['album']]
    except KeyError:
      mbid = 'NAN'
    player = [i['artist']['name'] for i in result['topalbums']['album']]
    song_name = [i['name'] for i in result['topalbums']['album']]

    df = pd.DataFrame({
        'Name' : [player],
        'MBID' : [mbid], # this not avaliable for all artist
        'playcount' : [playcount],
        'album_name': [song_name]

    })
    # temp_df = pd.DataFrame(df)
    album_df = pd.concat([album_df, df], ignore_index=True)
    return album_df

  x = most_played_album(result)



In [ ]:
album_df # this dataframe return playcount, and album of given artist
# i will use it to find similar albums and add those album here too
# making an album dataset ( final)
# and artist name i will use it to find similar artist and create an final artist dataframe


,Name,MBID,playcount,album_name
0,"[Nirvana, Nirvana, Nirvana, Nirvana, Nirvana, ...","[9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6, 9282c8b...","[49648238, 36656322, 44254569, 57283102, 25471...","[Nevermind, In Utero, MTV Unplugged in New Yor..."
1,"[The Killers, The Killers, The Killers, The Ki...","[95e1ead9-4d31-4808-a7ac-32c3614c116b, 95e1ead...","[95660952, 52756418, 29126702, 19626201, 63670...","[Hot Fuss, Sam's Town, Day & Age, Sawdust, Dir..."
2,"[Radiohead, Radiohead, Radiohead, Radiohead, R...","[a74b1b7f-71a5-4011-9441-d0b5e4122711, a74b1b7...","[156225375, 156990891, 58220470, 109970225, 81...","[OK Computer, In Rainbows, Pablo Honey, The Be..."
3,"[Gorillaz, Gorillaz, Gorillaz, Gorillaz, Goril...","[e21857d5-3256-4547-afb3-4b6ded592596, e21857d...","[99591086, 46933712, 73931495, 25149245, 13916...","[Demon Days, Gorillaz, Plastic Beach, Humanz (..."
4,"[Coldplay, Coldplay, Coldplay, Coldplay, Coldp...","[cc197bad-dc9c-440d-a5b5-d52ba2e14234, cc197ba...","[89429985, 75932244, 46244549, 42911011, 46871...","[Parachutes, A Rush of Blood to the Head, Viva..."
...,...,...,...,...
471,"[New Order, New Order, New Order, New Order, N...","[f1106b17-dcbb-45f6-b938-199ccfab50cc, f1106b1...","[9027380, 4947915, 6204017, 5256851, 1830969, ...","[Substance, The Best of New Order, Power corru..."
472,"[Men at Work, Men at Work, Men at Work, Men at...","[395cc503-63b5-4a0b-a20a-604e3fcacea2, 395cc50...","[7547003, 2855713, 1568684, 699548, 780738, 44...","[Business As Usual, The Best Of Men At Work: C..."
473,"[Dexys Midnight Runners, Dexys Midnight Runner...","[ccce2053-7007-4c36-b1e7-f8fcf5023a12, ccce205...","[4664890, 881845, 709405, 918676, 469248, 1451...","[Too Rye Ay, Come On Eileen / Dubious, Too Rye..."
474,"[Walk the Moon, Walk the Moon, Walk the Moon, ...","[d4aad415-9cd0-4845-9b05-0416fdcc9fc4, d4aad41...","[10126270, 6532915, 2177204, 767152, 680070, 3...","[Talking Is Hard, Walk the Moon, What If Nothi..."


artist.getInfo - Summary , Similar artist, tags, play count, listner count, mid, streamable?

In [ ]:
r = lastfm_get({
      'method': 'artist.getTopAlbums',
      'artist': 'Taylor Swift',
      # 'limit':  10
      # 'track': 'Blank Space'
  })

def jprint(obj):
    global result
    result = obj
    result = r.json()

jprint(r.json())

In [ ]:
import pandas as pd
def extract_artist_details(result):
  name = result['artist']['name']
  mbid = result['artist']['mbid']
  similar_artist = [i['name'] for i in result['artist']['similar']['artist']]
  listeners = result['artist']['stats']['listeners']
  play_counts = result['artist']['stats']['playcount']
  tags = [i['name'] for i in result['artist']['tags']['tag']]
  summary = result['artist']['bio']['summary']

  df = pd.DataFrame({
      'Name' : name,
      'MBID' : mbid,
      'similar_artist' : [similar_artist],
      'listeners':  listeners,
      'play_counts': play_counts,
      'tags': [tags],
      'summary': summary
  })

  return df


In [ ]:
artist_df = extract_artist_details(result)

In [ ]:
artist_df

,Name,MBID,similar_artist,listeners,play_counts,tags,summary
0,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,"[Sabrina Carpenter, Olivia Rodrigo, Gracie Abr...",4867742,2675958319,"[country, pop, female vocalists, singer-songwr...",Taylor Alison Swift is an American singer-song...


artist.getSimilar ---> match rate, mbid, name, streamable, url

In [ ]:
def get_similar_artist(result):
  name_of_artist = [i['name'] for i in result['similarartists']['artist']]
  # mbid = [i['mbid'] for i in result['similarartists']['artist']]
  match_score = [i['match'] for i in result['similarartists']['artist']]

  df = pd.DataFrame({
      'Name' : name_of_artist,
      # 'MBID' : mbid, # this not avaliable for all artist
      'match_score' : match_score,

  })

  return df


In [ ]:
similar_artist = get_similar_artist(result)

In [ ]:
similar_artist

,Name,match_score
0,Sabrina Carpenter,1
1,Olivia Rodrigo,0.999477
2,Gracie Abrams,0.874054
3,Chappell Roan,0.491840
4,Lorde,0.419973
5,Maisie Peters,0.383918
6,Harry Styles,0.366614
7,Kelsea Ballerini,0.358515
8,Dua Lipa,0.352020
9,Selena Gomez,0.351626


In [ ]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "similarartists": {
        "@attr": {
            "artist": "Taylor Swift"
        },
        "artist": [
            {
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "large"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "extralarge"
                    },
                    {
             

getTopTags - return top tags given by one user

In [ ]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "topalbums": {
        "@attr": {
            "artist": "Taylor Swift",
            "page": "1",
            "perPage": "50",
            "total": "91715",
            "totalPages": "1835"
        },
        "album": [
            {
                "artist": {
                    "mbid": "20244d07-534f-4eff-b4d4-930878889970",
                    "name": "Taylor Swift",
                    "url": "https://www.last.fm/music/Taylor+Swift"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/d3f083370c371a3ba1cddafaf193c27d.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/d3f083370c371a3ba1cddafaf193c27d.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/d3f0833

getTopAlbums - return playcount, name of album, mid

In [ ]:
result

{'toptracks': {'track': [{'name': 'Blank Space',
    'playcount': '22536275',
    'listeners': '1419150',
    'mbid': 'c3fe7791-0a91-4f0a-a89b-b056f38d3cde',
    'url': 'https://www.last.fm/music/Taylor+Swift/_/Blank+Space',
    'streamable': '0',
    'artist': {'name': 'Taylor Swift',
     'mbid': '20244d07-534f-4eff-b4d4-930878889970',
     'url': 'https://www.last.fm/music/Taylor+Swift'},
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'}],
    '@attr': {'rank': '1'}},
   {'name': 'Cruel Summer',
    'playcount': '39265917',
    'listeners

In [ ]:
def most_played_album(result):
  playcount = [i['playcount'] for i in result['topalbums']['album']]
  mbid = [i['artist']['mbid'] for i in result['topalbums']['album']]
  player = [i['artist']['name'] for i in result['topalbums']['album']]
  song_name = [i['name'] for i in result['topalbums']['album']]

  df = pd.DataFrame({
      'Name' : player,
      'MBID' : mbid, # this not avaliable for all artist
      'playcount' : playcount,
      'song_name': song_name

  })

  return df
# getTopAlbums


In [ ]:
popular_album = most_played_album(result)

In [ ]:
popular_album

,Name,MBID,playcount,song_name
0,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,231639366,Lover
1,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,202988822,Reputation
2,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,194548415,FOlKlORE
3,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,49965266,1989
4,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,117728759,Midnights
5,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,27966042,Fearless
6,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,86541192,1989 (Deluxe)
7,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,188341595,RED (Taylor's Version)
8,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,49391136,Speak Now
9,Taylor Swift,20244d07-534f-4eff-b4d4-930878889970,116211830,evermore


In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())

{
    "topalbums": {
        "@attr": {
            "artist": "Taylor Swift",
            "page": "1",
            "perPage": "50",
            "total": "91715",
            "totalPages": "1835"
        },
        "album": [
            {
                "artist": {
                    "mbid": "20244d07-534f-4eff-b4d4-930878889970",
                    "name": "Taylor Swift",
                    "url": "https://www.last.fm/music/Taylor+Swift"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/d3f083370c371a3ba1cddafaf193c27d.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/d3f083370c371a3ba1cddafaf193c27d.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/d3f0833

getTopTags - return count and name of tag

In [ ]:
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())


{
    "toptags": {
        "@attr": {
            "artist": "Taylor Swift"
        },
        "tag": [
            {
                "count": 100,
                "name": "country",
                "url": "https://www.last.fm/tag/country"
            },
            {
                "count": 75,
                "name": "pop",
                "url": "https://www.last.fm/tag/pop"
            },
            {
                "count": 60,
                "name": "female vocalists",
                "url": "https://www.last.fm/tag/female+vocalists"
            },
            {
                "count": 42,
                "name": "singer-songwriter",
                "url": "https://www.last.fm/tag/singer-songwriter"
            },
            {
                "count": 31,
                "name": "acoustic",
                "url": "https://www.last.fm/tag/acoustic"
            },
            {
                "count": 14,
                "name": "taylor swift",
                "url": "https:/

getTopTracks - listeners, mbid, name of song, playcount, streamble

In [ ]:
def get_top_tracks(result):
  song_name = [i['name'] for i in result['toptracks']['track']]
  artist_name = [i['artist']['name'] for i in result['toptracks']['track']]
  playcount = [ i['playcount'] for i in result['toptracks']['track']]
  listners = [ i['listeners'] for i in result['toptracks']['track']]

  df = pd.DataFrame({
      'Name' : artist_name,
      'song_name' : song_name,
      'playcount' :playcount,

  })

  return df

# getTopTracks


In [ ]:
top_tracks = get_top_tracks(result)

In [ ]:
top_tracks

,Name,song_name,playcount
0,Taylor Swift,Blank Space,22541417
1,Taylor Swift,Cruel Summer,39283694
2,Taylor Swift,Shake It Off,15422316
3,Taylor Swift,carDIGan,34587990
4,Taylor Swift,Style,25572048
5,Taylor Swift,Love Story,10024506
6,Taylor Swift,Anti-Hero,28008481
7,Taylor Swift,You Belong with Me,9517179
8,Taylor Swift,aUgUSt,32758760
9,Taylor Swift,Lover,21736710


In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())

{
    "toptracks": {
        "@attr": {
            "artist": "Taylor Swift",
            "page": "1",
            "perPage": "50",
            "total": "161682",
            "totalPages": "3234"
        },
        "track": [
            {
                "@attr": {
                    "rank": "1"
                },
                "artist": {
                    "mbid": "20244d07-534f-4eff-b4d4-930878889970",
                    "name": "Taylor Swift",
                    "url": "https://www.last.fm/music/Taylor+Swift"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
     

artist.search - return the artist and other artist with whom they usually collaborate

In [ ]:
def most_related_artist(result):
  similar_artist = [i['name'] for i in result['results']['artistmatches']['artist']]
  df = pd.DataFrame({
      'similar_artist' : similar_artist
      })

  return df

In [ ]:
related_artist = [most_related_artist(result)]

In [ ]:
related_artist

[                                     similar_artist
 0                                      Taylor Swift
 1                         Taylor Swift, Post Malone
 2                            Taylor Swift, Bon Iver
 3                        Taylor Swift, Lana Del Rey
 4              Taylor Swift, Florence + The Machine
 5                  Taylor Swift, Ed Sheeran, Future
 6                                Taylor Swift, HAIM
 7                               ZAYN & Taylor Swift
 8                        Taylor Swift, Fall Out Boy
 9                        Taylor Swift, The National
 10                    Taylor Swift, Phoebe Bridgers
 11                     Taylor Swift, Kendrick Lamar
 12                          Taylor Swift, Ice Spice
 13                    Taylor Swift, Hayley Williams
 14                               Zayn, Taylor Swift
 15                         Taylor Swift, Ed Sheeran
 16                     Taylor Swift, Gary Lightbody
 17                    Taylor Swift, Chris Sta

In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())


{
    "results": {
        "@attr": {
            "for": "Taylor Swift"
        },
        "artistmatches": {
            "artist": [
                {
                    "image": [
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                            "size": "small"
                        },
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                            "size": "medium"
                        },
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                            "size": "large"
                        },
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                  

In [ ]:
r = lastfm_get({
    'method': 'track.search',
    'artist': 'Taylor Swift',
    # 'limit':  10
    'track': 'Blank Space'
})

r.status_code

200

track.getInfo - position, artist, toptags for that song, playcount, album of song,

In [ ]:
def get_similar_songs(result):
  name_of_artist = [i['artist']['name'] for i in result['similartracks']['track']]
  # mbid_song = [i['mbid'] for i in result['similartracks']['track']]
  match_score = [i['match'] for i in result['similartracks']['track']]
  song_name = [i['name'] for i in result['similartracks']['track']]
  playcount = [i['playcount'] for i in result['similartracks']['track']]

  df = pd.DataFrame({
      'Name' : name_of_artist,
      #'mbid_of_artist': mbid_song, # mbid - this is nan in some cases
      'song_name' : song_name,
      'match_score': match_score,
      'playcount' :playcount,


  })

  return df


In [ ]:
similar_songs = get_similar_songs(result)

In [ ]:
similar_songs

,Name,song_name,match_score,playcount
0,Taylor Swift,Welcome to New York,1.000000,7176801
1,Taylor Swift,Style,0.981254,25572048
2,Ariana Grande,One Last Time,0.274156,12020581
3,Miley Cyrus,Wrecking Ball,0.263028,8354026
4,Ellie Goulding,Love Me Like You Do,0.257751,3155875
...,...,...,...,...
95,Olivia Rodrigo,deja vu,0.011114,24038435
96,Harry Styles,As It Was,0.010063,33451448
97,Britney Spears,Toxic,0.008802,19780428
98,Britney Spears,...Baby One More Time,0.008739,11117509


In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())


{
    "toptags": {
        "@attr": {
            "artist": "Taylor Swift",
            "track": "Blank Space"
        },
        "tag": [
            {
                "count": 100,
                "name": "pop",
                "url": "https://www.last.fm/tag/pop"
            },
            {
                "count": 43,
                "name": "love at first listen",
                "url": "https://www.last.fm/tag/love+at+first+listen"
            },
            {
                "count": 32,
                "name": "female vocalists",
                "url": "https://www.last.fm/tag/female+vocalists"
            },
            {
                "count": 32,
                "name": "2014",
                "url": "https://www.last.fm/tag/2014"
            },
            {
                "count": 23,
                "name": "best of 2014",
                "url": "https://www.last.fm/tag/best+of+2014"
            },
            {
                "count": 17,
                "name": "el

getSimilar - gives song similar to current track

In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())


{
    "similarartists": {
        "@attr": {
            "artist": "Taylor Swift"
        },
        "artist": [
            {
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "medium"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "large"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png",
                        "size": "extralarge"
                    },
                    {
             

getTopTags - give top tags for current song

In [ ]:
def get_tag_for_song(result):
  top_tags = [i['name'] for i in result['toptags']['tag']]
  return top_tags

In [ ]:
song_tags = get_tag_for_song(result)


In [ ]:
song_tags

['pop',
 'love at first listen',
 'female vocalists',
 '2014',
 'best of 2014',
 'electropop',
 'taylor swift',
 '10s',
 'synthpop',
 'max martin',
 '2010s',
 'sunflower',
 'fav888',
 'my dance room',
 'chillout',
 'world',
 '3',
 'World Music',
 '1989',
 'Skitz0rz',
 'uitluisteren artist',
 'holloween skitz0rz',
 'songs that i love the lyric',
 'chill',
 'Love',
 'overrated',
 'american',
 'female vocalist',
 'alt-pop',
 'blank space',
 'seen live',
 'guilty pleasure',
 'fav',
 '9 of 10 stars',
 'Love it',
 'Masterpiece',
 '<3',
 'pop perfection',
 'contemporary rnb',
 'us number one',
 'you look like my next mistake',
 'melhor que a carreira toda de katy perry',
 'electronic',
 'indie',
 'female',
 'indie pop',
 'electro',
 'pop rock',
 'canada',
 'happy',
 'Queen',
 'solo',
 'catchy',
 'Contemporary',
 'soft rock',
 'adult contemporary',
 'romantic',
 'feel good',
 'synth',
 'Radio',
 'party',
 'amazing',
 'rnb',
 'sex',
 'female vocals',
 'cool',
 'vocal harmonies',
 'best song eve

In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())


{
    "toptags": {
        "@attr": {
            "artist": "Taylor Swift",
            "track": "Blank Space"
        },
        "tag": [
            {
                "count": 100,
                "name": "pop",
                "url": "https://www.last.fm/tag/pop"
            },
            {
                "count": 43,
                "name": "love at first listen",
                "url": "https://www.last.fm/tag/love+at+first+listen"
            },
            {
                "count": 32,
                "name": "female vocalists",
                "url": "https://www.last.fm/tag/female+vocalists"
            },
            {
                "count": 32,
                "name": "2014",
                "url": "https://www.last.fm/tag/2014"
            },
            {
                "count": 23,
                "name": "best of 2014",
                "url": "https://www.last.fm/tag/best+of+2014"
            },
            {
                "count": 17,
                "name": "el

In [ ]:
import json
result = {}
def jprint(obj):
    global result
    result = obj
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    #result = text
    print(text)

jprint(r.json())


{
    "results": {
        "@attr": {},
        "opensearch:Query": {
            "#text": "",
            "role": "request",
            "startPage": "1"
        },
        "opensearch:itemsPerPage": "30",
        "opensearch:startIndex": "0",
        "opensearch:totalResults": "2582",
        "trackmatches": {
            "track": [
                {
                    "artist": "Taylor Swift",
                    "image": [
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png",
                            "size": "small"
                        },
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png",
                            "size": "medium"
                        },
                        {
                            "#text": "https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821

# Getting the artist info for artist mentioned in list

In [ ]:
artist_info = pd.DataFrame()

In [ ]:
df['artist']

,artist
0,Nirvana
1,The Killers
2,Radiohead
4,Gorillaz
5,Coldplay
...,...
2475,New Order
2477,Men at Work
2478,Dexys Midnight Runners
2484,Walk the Moon


In [ ]:
result

{'artist': {'name': 'Nirvana',
  'mbid': '9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6',
  'url': 'https://www.last.fm/music/Nirvana',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '0',
  'stats': {'listeners': '6294185', 'playcount': '418317105'},
  'similar': {'artist': [{'name': 'Kurt Cobain'

In [ ]:
df

,tag,track_name,artist
0,rock,Smells Like Teen Spirit,Nirvana
1,rock,Mr. Brightside,The Killers
2,rock,Creep,Radiohead
4,rock,Feel Good Inc.,Gorillaz
5,rock,Yellow,Coldplay
...,...,...,...
2475,new wave,Blue Monday,New Order
2477,new wave,Down Under,Men at Work
2478,new wave,Come on Eileen,Dexys Midnight Runners
2484,new wave,Shut Up and Dance,Walk the Moon


In [ ]:
import pandas as pd

def artist_complete_info(df, artist_info):
    # Function to handle API response and add to artist_info DataFrame
    def extract_artist_details(result, artist_info):
        try:
          name = result['artist']['name']
        except KeyError:
          name = 'NAN'
        try :
          mbid = result['artist']['mbid'] # Use .get() to handle missing keys
        except KeyError:
          mbid = 'NAN'
        try:
          similar_artist = [i['name'] for i in result['artist']['similar']['artist']]
        except KeyError:
          similar_artist = 'NAN'
        try :
          listeners = result['artist']['stats']['listeners']
        except KeyError:
          listeners = 'NAN'
        try:
          play_counts = result['artist']['stats']['playcount']
        except KeyError:
          play_counts = 'NAN'
        try:
          tags = [i['name'] for i in result['artist']['tags']['tag']]
        except KeyError:
          tags = 'NAN'
        try:
          summary = result['artist']['bio']['summary']
        except KeyError:
          summary = 'NAN'
        # Create a DataFrame with the new artist data
        artist_data = pd.DataFrame({
            'Name': [name],
            'MBID': [mbid],
            'similar_artist': [similar_artist],
            'listeners': [listeners],
            'play_counts': [play_counts],
            'tags': [tags],
            'summary': [summary]
        })

        # Check if MBID already exists in the current DataFrame
        if artist_info.empty or mbid not in artist_info['MBID'].values:
            # Concatenate the new artist data to artist_info DataFrame
            artist_info = pd.concat([artist_info, artist_data], ignore_index=True)

        # Always return artist_info
        return artist_info

    for artist_name in df:
        # Fetch artist data from API
        response = lastfm_get({'method': 'artist.getInfo', 'artist': artist_name})

        # Check if response is valid
        if response.status_code == 200:
            result = response.json()
            artist_info = extract_artist_details(result, artist_info)
        else:
            print(f"Failed to fetch data for artist: {artist_name}")

    return artist_info


In [ ]:
artist_info = artist_complete_info(df['artist'], artist_info)

In [ ]:
artist_info

,Name,MBID,similar_artist,listeners,play_counts,tags,summary
0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"[Kurt Cobain, Alice in Chains, Soundgarden, Th...",6297168,418658293,"[Grunge, rock, alternative, alternative rock, ...","There are multiple artists tracked as ""Nirvana..."
1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"[Brandon Flowers, Kings of Leon, Snow Patrol, ...",5950326,291647767,"[indie, rock, indie rock, alternative, alterna...",There are multiple artists using this name:\n\...
2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"[Thom Yorke, Atoms for Peace, Jeff Buckley, Th...",6815000,928426766,"[alternative, alternative rock, rock, indie, e...",Radiohead are an English rock band formed in A...
3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"[Damon Albarn, The Good, the Bad & the Queen, ...",5617912,363054163,"[alternative, electronic, Hip-Hop, rock, indie]",Gorillaz are a British virtual band created in...
4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"[Keane, Imagine Dragons, Snow Patrol, Travis, ...",7710436,592812260,"[rock, alternative, britpop, alternative rock,...",Coldplay are a britpop band formed in London i...
...,...,...,...,...,...,...,...
425,Faithless,23d9d74d-c95e-46a6-be26-a6075c49747a,"[Groove Armada, Leftfield, Energy 52, Underwor...",1575211,29668013,"[electronic, trip-hop, dance, House, electronica]","Formed in 1995 by Sister Bliss, Faithless was ..."
426,New Order,f1106b17-dcbb-45f6-b938-199ccfab50cc,"[Electronic, Joy Division, The Other Two, Orch...",2694948,92067307,"[new wave, electronic, post-punk, 80s, alterna...",New Order are a new wave/synthpop band which f...
427,Men at Work,395cc503-63b5-4a0b-a20a-604e3fcacea2,"[The Police, INXS, Midnight Oil, The Outfield,...",1554350,18023408,"[80s, new wave, rock, australian, pop]",Men at Work is a rock band that formed in Melb...
428,Dexys Midnight Runners,ccce2053-7007-4c36-b1e7-f8fcf5023a12,"[Fun Boy Three, Adam and the Ants, The Jam, Ma...",1302651,10809221,"[new wave, 80s, post-punk, british, pop]",Dexys Midnight Runners - the name consistently...


# i got similar artist here now i will extract it


In [ ]:
def get_similar_ar_list(df):
  similar_artist = artist_info['similar_artist']
  expanded_df = similar_artist.explode('similar_artist').reset_index(drop=True)
  expanded_df.dropna(inplace=True)
  expanded_df.drop_duplicates(keep='first')
  unique_artists = [artist for artist in expanded_df if artist not in artist_info['Name'].values]
  expanded_df_unique = pd.Series(unique_artists).drop_duplicates().reset_index(drop=True)
  return expanded_df_unique

In [ ]:
expanded_df_unique = get_similar_ar_list(artist_info)

In [ ]:
expanded_df_unique

,0
0,Kurt Cobain
1,Alice in Chains
2,The Jins
3,Mudhoney
4,Brandon Flowers
...,...
1311,Lloyd Cole and The Commotions
1312,Neon Trees
1313,Bad Suns
1314,Smallpools


In [ ]:
artist_info = artist_complete_info(expanded_df_unique, artist_info)

In [ ]:
artist_info

,Name,MBID,similar_artist,listeners,play_counts,tags,summary
0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"[Kurt Cobain, Alice in Chains, Soundgarden, Th...",6297168,418658293,"[Grunge, rock, alternative, alternative rock, ...","There are multiple artists tracked as ""Nirvana..."
1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"[Brandon Flowers, Kings of Leon, Snow Patrol, ...",5950326,291647767,"[indie, rock, indie rock, alternative, alterna...",There are multiple artists using this name:\n\...
2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"[Thom Yorke, Atoms for Peace, Jeff Buckley, Th...",6815000,928426766,"[alternative, alternative rock, rock, indie, e...",Radiohead are an English rock band formed in A...
3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"[Damon Albarn, The Good, the Bad & the Queen, ...",5617912,363054163,"[alternative, electronic, Hip-Hop, rock, indie]",Gorillaz are a British virtual band created in...
4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"[Keane, Imagine Dragons, Snow Patrol, Travis, ...",7710436,592812260,"[rock, alternative, britpop, alternative rock,...",Coldplay are a britpop band formed in London i...
...,...,...,...,...,...,...,...
1528,Lloyd Cole and The Commotions,7451c6e2-02f0-4158-904c-7f477683ce5e,"[Lloyd Cole, Aztec Camera, The Housemartins, T...",189390,2178120,"[80s, pop, new wave, alternative, indie]",Lloyd Cole and the Commotions was a British po...
1529,Neon Trees,16243662-8538-4746-a0fb-0d15b5828b8e,"[Walk the Moon, American Authors, The Mowgli's...",1674568,25535369,"[indie, indie rock, alternative, alternative r...",Neon Trees is an alternative rock/new wave ban...
1530,Bad Suns,95015b4c-1aef-4e28-9d36-c9546c194f0c,"[Coin, CRUISR, HUNNY, Smallpools, Hippo Campus]",570073,13529778,"[indie rock, indie, rock, seen live, alternative]",Southern California rock band from Woodland Hi...
1531,Smallpools,91db24e6-1c28-4761-b829-838e3c9155bc,"[Magic Man, CRUISR, The Colourist, The Griswol...",436708,6238040,"[indie pop, indie, pop rock, alternative, seen...",Smallpools is an indie pop/alternative band fr...


In [ ]:
expanded_df_unique_second_iteration = get_similar_ar_list(artist_info)

In [ ]:
expanded_df_unique_second_iteration

,0
0,The Jins
1,twenty one pilots
2,FFS
3,Daron Malakian and Scars on Broadway
4,Dead by Sunrise
...,...
2417,CRUISR
2418,The Griswolds
2419,Flor
2420,Parade of Lights


In [ ]:
artist_info = artist_complete_info(expanded_df_unique_second_iteration, artist_info)

In [ ]:
artist_info

,Name,MBID,similar_artist,listeners,play_counts,tags,summary
0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"[Kurt Cobain, Alice in Chains, Soundgarden, Th...",6297168,418658293,"[Grunge, rock, alternative, alternative rock, ...","There are multiple artists tracked as ""Nirvana..."
1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"[Brandon Flowers, Kings of Leon, Snow Patrol, ...",5950326,291647767,"[indie, rock, indie rock, alternative, alterna...",There are multiple artists using this name:\n\...
2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"[Thom Yorke, Atoms for Peace, Jeff Buckley, Th...",6815000,928426766,"[alternative, alternative rock, rock, indie, e...",Radiohead are an English rock band formed in A...
3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"[Damon Albarn, The Good, the Bad & the Queen, ...",5617912,363054163,"[alternative, electronic, Hip-Hop, rock, indie]",Gorillaz are a British virtual band created in...
4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"[Keane, Imagine Dragons, Snow Patrol, Travis, ...",7710436,592812260,"[rock, alternative, britpop, alternative rock,...",Coldplay are a britpop band formed in London i...
...,...,...,...,...,...,...,...
3384,CRUISR,3fa32017-a163-450b-9b3b-b210100c09a8,"[Wild Party, Smallpools, Bad Suns, Coin, Flor]",140686,1370211,"[indie pop, indie, pop, indie rock, USA]",CRUISR began in 2012 as the solo project of An...
3385,The Griswolds,f7547a5b-df52-4c07-b3ca-fb72caff512c,"[Smallpools, The Colourist, Magic Man, Fmlybnd...",278873,2387818,"[indie, indie pop, seen live, australian, elec...",There are two bands called The Griswolds: (1) ...
3386,Flor,70ac7e65-fe55-49af-8f4c-aab263d4a61e,"[Smallpools, Nightly, The Band CAMINO, Coin, B...",268541,5174219,"[indie, seen live, indie pop, alternative, pop]",Three different artists are under this profile...
3387,Parade of Lights,ebfbe426-7b2a-40de-96c9-24285d648cf1,"[Castlecomer, Magic Man, Coasts, Penguin Priso...",99714,815112,"[electronic, indie, rock, indie pop, indie rock]",Parade of Lights are an alt-electro band based...


In [ ]:
artist_info.to_csv('/content/artist_info.csv')

In [ ]:
r = lastfm_get({
      'method': 'artist.getSimilar',
      'artist': 'Nirvana',
      'limit': 500
      # 'limit':  10
      # 'track': 'Blank Space'
  })

def jprint(obj):
    global result
    result = obj
    result = r.json()

jprint(r.json())

In [ ]:
artist_name = artist_info['Name']

In [ ]:
artist_name

,Name
0,Nirvana
1,The Killers
2,Radiohead
3,Gorillaz
4,Coldplay
...,...
3384,CRUISR
3385,The Griswolds
3386,Flor
3387,Parade of Lights


In [ ]:
import pandas as pd

def similarity(names):
    # Initialize an empty DataFrame to store results
    simlarity_df = pd.DataFrame()

    # Function to handle API response and add to the DataFrame
    def get_similar_artist(result):
        name_of_artist = [i['name'] for i in result['similarartists']['artist']]
        match_score = [i['match'] for i in result['similarartists']['artist']]

        # Create a temporary DataFrame with the fetched data
        df = pd.DataFrame({
            'Name': name_of_artist,
            'match_score': match_score,
        })

        return df

    # Iterate over each artist name
    for artist_name in names:
        # Fetch artist data from API
        response = lastfm_get({'method': 'artist.getSimilar', 'artist': artist_name})

        # Check if response is valid
        if response.status_code == 200:
            result = response.json()
            # Get similar artists and append to simlarity_df
            simlarity_df = pd.concat([simlarity_df, get_similar_artist(result)], ignore_index=True)
        else:
            print(f"Failed to fetch data for artist: {artist_name}")

    return simlarity_df


In [ ]:
result = similarity(artist_info['Name'])

In [ ]:
result

,Name,match_score
0,Kurt Cobain,1
1,Alice in Chains,0.564123
2,Soundgarden,0.377042
3,The Jins,0.343049
4,Mudhoney,0.320566
...,...,...
334295,Bad Bad Hats,0.145331
334296,Ladyhawke,0.144311
334297,The Jungle Giants,0.142913
334298,A Great Big World,0.142557


In [ ]:
result

,Name,match_score
0,Kurt Cobain,1
1,Alice in Chains,0.564123
2,Soundgarden,0.377042
3,The Jins,0.343049
4,Mudhoney,0.320566
...,...,...
334295,Bad Bad Hats,0.145331
334296,Ladyhawke,0.144311
334297,The Jungle Giants,0.142913
334298,A Great Big World,0.142557


In [ ]:
result['match_score'] = pd.to_numeric(result['match_score'], errors = 'coerce')

In [ ]:
result = result[result['match_score'] < 0.1]

In [ ]:
result

,Name,match_score
49,Melvins,0.095164
50,Seven Mary Three,0.094937
51,Queens of the Stone Age,0.094560
52,Soul Asylum,0.091110
53,Oasis,0.090931
...,...,...
333395,David Sylvian,0.087432
333396,The Pretenders,0.087353
333397,Steve Wynn,0.087239
333398,Madness,0.086667


In [ ]:
artist_info = artist_complete_info(result['Name'], artist_info)

In [ ]:
artist_info

,Name,MBID,similar_artist,listeners,play_counts,tags,summary
0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"[Kurt Cobain, Alice in Chains, Soundgarden, Th...",6297168,418658293,"[Grunge, rock, alternative, alternative rock, ...","There are multiple artists tracked as ""Nirvana..."
1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"[Brandon Flowers, Kings of Leon, Snow Patrol, ...",5950326,291647767,"[indie, rock, indie rock, alternative, alterna...",There are multiple artists using this name:\n\...
2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"[Thom Yorke, Atoms for Peace, Jeff Buckley, Th...",6815000,928426766,"[alternative, alternative rock, rock, indie, e...",Radiohead are an English rock band formed in A...
3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"[Damon Albarn, The Good, the Bad & the Queen, ...",5617912,363054163,"[alternative, electronic, Hip-Hop, rock, indie]",Gorillaz are a British virtual band created in...
4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"[Keane, Imagine Dragons, Snow Patrol, Travis, ...",7710436,592812260,"[rock, alternative, britpop, alternative rock,...",Coldplay are a britpop band formed in London i...
...,...,...,...,...,...,...,...
11772,The Ordinary Boys,69b50aed-f122-4c45-aba9-863043f5b513,"[The Futureheads, Hard-Fi, The Enemy, Little M...",216327,1930787,"[indie, indie rock, british, ska, rock]",The Ordinary Boys are an English indie rock ba...
11773,Rhoda Dakar,5d678a86-6a6b-49a5-9bb7-0b19c4999859,"[Neville Staple, Dave & Ansell Collins, Two To...",4600,27354,"[reggae, ska, british, female vocalists, rocks...",Rhoda Dakar (born 1959) is a British singer an...
11774,The Smithereens,fae2cd4c-f847-4cb3-b311-60c8906cdb0b,"[Guadalcanal Diary, BoDeans, Marshall Crenshaw...",144711,1383566,"[power pop, rock, 80s, alternative, college rock]",The Smithereens are a power pop/alternative ro...
11775,Gruff Rhys,254caf14-7b63-47d1-aa9f-acc7cb541814,"[Super Furry Animals, Bill Ryder-Jones, Nadine...",136720,2293498,"[indie, seen live, welsh, singer-songwriter, i...",Gruff Rhys is the lead singer of the Welsh ban...


In [ ]:
artist_info.to_csv('/content/artist_info.csv')

In [ ]:
r = lastfm_get({
      'method': 'artist.getTopAlbums',
      'artist': 'Taylor Swift',
      # 'limit':  10
      # 'track': 'Blank Space'
  })

def jprint(obj):
    global result
    result = obj
    result = r.json()

jprint(r.json())

In [ ]:
result

{'topalbums': {'album': [{'name': 'Lover',
    'playcount': 231966423,
    'url': 'https://www.last.fm/music/Taylor+Swift/Lover',
    'artist': {'name': 'Taylor Swift',
     'mbid': '20244d07-534f-4eff-b4d4-930878889970',
     'url': 'https://www.last.fm/music/Taylor+Swift'},
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/d3f083370c371a3ba1cddafaf193c27d.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/d3f083370c371a3ba1cddafaf193c27d.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/d3f083370c371a3ba1cddafaf193c27d.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/d3f083370c371a3ba1cddafaf193c27d.png',
      'size': 'extralarge'}]},
   {'name': 'Reputation',
    'playcount': 203316416,
    'url': 'https://www.last.fm/music/Taylor+Swift/Reputation',
    'artist': {'name': 'Taylor Swift',
     'mbid': '20244d07-534f-4eff-b4d4-930878889970',
     'u

In [ ]:
import pandas as pd
artist_info = pd.read_csv('/content/artist_info (1).csv')

In [ ]:
artist_info

,Unnamed: 0,Name,MBID,similar_artist,listeners,play_counts,tags,summary,top_album
0,0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"['Kurt Cobain', 'Alice in Chains', 'Soundgarde...",6297168,418658293,"['Grunge', 'rock', 'alternative', 'alternative...","There are multiple artists tracked as ""Nirvana...",nan
1,1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"['Brandon Flowers', 'Kings of Leon', 'Snow Pat...",5950326,291647767,"['indie', 'rock', 'indie rock', 'alternative',...",There are multiple artists using this name:\n\...,nan
2,2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"['Thom Yorke', 'Atoms for Peace', 'Jeff Buckle...",6815000,928426766,"['alternative', 'alternative rock', 'rock', 'i...",Radiohead are an English rock band formed in A...,nan
3,3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"['Damon Albarn', 'The Good, the Bad & the Quee...",5617912,363054163,"['alternative', 'electronic', 'Hip-Hop', 'rock...",Gorillaz are a British virtual band created in...,nan
4,4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"['Keane', 'Imagine Dragons', 'Snow Patrol', 'T...",7710436,592812260,"['rock', 'alternative', 'britpop', 'alternativ...",Coldplay are a britpop band formed in London i...,nan
...,...,...,...,...,...,...,...,...,...
11772,11772,The Ordinary Boys,69b50aed-f122-4c45-aba9-863043f5b513,"['The Futureheads', 'Hard-Fi', 'The Enemy', 'L...",216327,1930787,"['indie', 'indie rock', 'british', 'ska', 'rock']",The Ordinary Boys are an English indie rock ba...,nan
11773,11773,Rhoda Dakar,5d678a86-6a6b-49a5-9bb7-0b19c4999859,"['Neville Staple', 'Dave & Ansell Collins', 'T...",4600,27354,"['reggae', 'ska', 'british', 'female vocalists...",Rhoda Dakar (born 1959) is a British singer an...,nan
11774,11774,The Smithereens,fae2cd4c-f847-4cb3-b311-60c8906cdb0b,"['Guadalcanal Diary', 'BoDeans', 'Marshall Cre...",144711,1383566,"['power pop', 'rock', '80s', 'alternative', 'c...",The Smithereens are a power pop/alternative ro...,nan
11775,11775,Gruff Rhys,254caf14-7b63-47d1-aa9f-acc7cb541814,"['Super Furry Animals', 'Bill Ryder-Jones', 'N...",136720,2293498,"['indie', 'seen live', 'welsh', 'singer-songwr...",Gruff Rhys is the lead singer of the Welsh ban...,nan


In [ ]:
artist_info.drop(columns = ['Unnamed: 0'], inplace =True)

In [ ]:
def get_top_albums(names, artist_info):


    def most_played_album(result):

      # playcount = [i['playcount'] for i in result['topalbums']['album']]
      # mbid = [i['artist']['mbid'] for i in result['topalbums']['album']]
      # player = [i['artist']['name'] for i in result['topalbums']['album']]
      song_name = [i['name'] for i in result['topalbums']['album']]


      return song_name

    # Iterate over each artist name
    for artist_name in names:
        # Fetch artist data from API
        response = lastfm_get({'method': 'artist.getTopAlbums', 'artist': artist_name})

        # Check if response is valid
        if response.status_code == 200:
            result = response.json()
            top_albums = most_played_album(result)
            artist_info.loc[artist_info['Name'] == artist_name ,'top_album'] = ', '.join(top_albums)

        else:
            print(f"Failed to fetch data for artist: {artist_name}")

    return artist_info


In [ ]:
artist_info = get_top_albums(artist_info['Name'].to_list(), artist_info)

In [ ]:
artist_info

,Unnamed: 0,Name,MBID,similar_artist,listeners,play_counts,tags,summary,top_album
0,0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"['Kurt Cobain', 'Alice in Chains', 'Soundgarde...",6297168,418658293,"['Grunge', 'rock', 'alternative', 'alternative...","There are multiple artists tracked as ""Nirvana...","Nevermind, In Utero, MTV Unplugged in New York..."
1,1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"['Brandon Flowers', 'Kings of Leon', 'Snow Pat...",5950326,291647767,"['indie', 'rock', 'indie rock', 'alternative',...",There are multiple artists using this name:\n\...,"Hot Fuss, Sam's Town, Day & Age, Sawdust, Dire..."
2,2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"['Thom Yorke', 'Atoms for Peace', 'Jeff Buckle...",6815000,928426766,"['alternative', 'alternative rock', 'rock', 'i...",Radiohead are an English rock band formed in A...,"OK Computer, In Rainbows, Pablo Honey, The Ben..."
3,3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"['Damon Albarn', 'The Good, the Bad & the Quee...",5617912,363054163,"['alternative', 'electronic', 'Hip-Hop', 'rock...",Gorillaz are a British virtual band created in...,"Demon Days, Gorillaz, Plastic Beach, Humanz (D..."
4,4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"['Keane', 'Imagine Dragons', 'Snow Patrol', 'T...",7710436,592812260,"['rock', 'alternative', 'britpop', 'alternativ...",Coldplay are a britpop band formed in London i...,"Parachutes, A Rush of Blood to the Head, Viva ..."
...,...,...,...,...,...,...,...,...,...
11772,11772,The Ordinary Boys,69b50aed-f122-4c45-aba9-863043f5b513,"['The Futureheads', 'Hard-Fi', 'The Enemy', 'L...",216327,1930787,"['indie', 'indie rock', 'british', 'ska', 'rock']",The Ordinary Boys are an English indie rock ba...,"Over the Counter Culture, Brassbound, How To G..."
11773,11773,Rhoda Dakar,5d678a86-6a6b-49a5-9bb7-0b19c4999859,"['Neville Staple', 'Dave & Ansell Collins', 'T...",4600,27354,"['reggae', 'ska', 'british', 'female vocalists...",Rhoda Dakar (born 1959) is a British singer an...,"The Boiler, Sings The Bodysnatchers, Version G..."
11774,11774,The Smithereens,fae2cd4c-f847-4cb3-b311-60c8906cdb0b,"['Guadalcanal Diary', 'BoDeans', 'Marshall Cre...",144711,1383566,"['power pop', 'rock', '80s', 'alternative', 'c...",The Smithereens are a power pop/alternative ro...,"Especially For You, Smithereens 11, Green Thou..."
11775,11775,Gruff Rhys,254caf14-7b63-47d1-aa9f-acc7cb541814,"['Super Furry Animals', 'Bill Ryder-Jones', 'N...",136720,2293498,"['indie', 'seen live', 'welsh', 'singer-songwr...",Gruff Rhys is the lead singer of the Welsh ban...,"Hotel Shampoo, American Interior, Candylion, S..."


In [ ]:
artist_info['top_song'] = 'NAN'

In [ ]:
artist_info

,Name,MBID,similar_artist,listeners,play_counts,tags,summary,top_album,top_song
0,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,"['Kurt Cobain', 'Alice in Chains', 'Soundgarde...",6297168,418658293,"['Grunge', 'rock', 'alternative', 'alternative...","There are multiple artists tracked as ""Nirvana...","Nevermind, In Utero, MTV Unplugged in New York...",NAN
1,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,"['Brandon Flowers', 'Kings of Leon', 'Snow Pat...",5950326,291647767,"['indie', 'rock', 'indie rock', 'alternative',...",There are multiple artists using this name:\n\...,"Hot Fuss, Sam's Town, Day & Age, Sawdust, Dire...",NAN
2,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,"['Thom Yorke', 'Atoms for Peace', 'Jeff Buckle...",6815000,928426766,"['alternative', 'alternative rock', 'rock', 'i...",Radiohead are an English rock band formed in A...,"OK Computer, In Rainbows, Pablo Honey, The Ben...",NAN
3,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,"['Damon Albarn', 'The Good, the Bad & the Quee...",5617912,363054163,"['alternative', 'electronic', 'Hip-Hop', 'rock...",Gorillaz are a British virtual band created in...,"Demon Days, Gorillaz, Plastic Beach, Humanz (D...",NAN
4,Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,"['Keane', 'Imagine Dragons', 'Snow Patrol', 'T...",7710436,592812260,"['rock', 'alternative', 'britpop', 'alternativ...",Coldplay are a britpop band formed in London i...,"Parachutes, A Rush of Blood to the Head, Viva ...",NAN
...,...,...,...,...,...,...,...,...,...
11772,The Ordinary Boys,69b50aed-f122-4c45-aba9-863043f5b513,"['The Futureheads', 'Hard-Fi', 'The Enemy', 'L...",216327,1930787,"['indie', 'indie rock', 'british', 'ska', 'rock']",The Ordinary Boys are an English indie rock ba...,"Over the Counter Culture, Brassbound, How To G...",NAN
11773,Rhoda Dakar,5d678a86-6a6b-49a5-9bb7-0b19c4999859,"['Neville Staple', 'Dave & Ansell Collins', 'T...",4600,27354,"['reggae', 'ska', 'british', 'female vocalists...",Rhoda Dakar (born 1959) is a British singer an...,"The Boiler, Sings The Bodysnatchers, Version G...",NAN
11774,The Smithereens,fae2cd4c-f847-4cb3-b311-60c8906cdb0b,"['Guadalcanal Diary', 'BoDeans', 'Marshall Cre...",144711,1383566,"['power pop', 'rock', '80s', 'alternative', 'c...",The Smithereens are a power pop/alternative ro...,"Especially For You, Smithereens 11, Green Thou...",NAN
11775,Gruff Rhys,254caf14-7b63-47d1-aa9f-acc7cb541814,"['Super Furry Animals', 'Bill Ryder-Jones', 'N...",136720,2293498,"['indie', 'seen live', 'welsh', 'singer-songwr...",Gruff Rhys is the lead singer of the Welsh ban...,"Hotel Shampoo, American Interior, Candylion, S...",NAN


In [ ]:
import pandas as pd

def get_top_songs(names, artist_info):


    def get_top_tracks(result):
      song_name = [i['name'] for i in result['toptracks']['track']]


      return song_name

    # Iterate over each artist name
    for artist_name in names:
        # Fetch artist data from API
        response = lastfm_get({'method': 'artist.getTopTracks', 'artist': artist_name})

        # Check if response is valid
        if response.status_code == 200:
            result = response.json()
            top_songs = [i['name'] for i in result['toptracks']['track']]
            artist_info.loc[artist_info['Name'] == artist_name ,'top_songs'] = ', '.join(top_songs)

        else:
            print(f"Failed to fetch data for artist: {artist_name}")

    return artist_info


In [ ]:
artist_info.to_csv('/content/artist_info.csv')

In [ ]:
artist_info = get_top_songs(artist_info['Name'].tolist(), artist_info)

In [ ]:
artist_info.to_csv('/content/artist_info.csv')